In [18]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from typing_extensions import Concatenate
from langchain_openai import ChatOpenAI
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
import os
from datetime import datetime
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Access the environment variables
api_key = os.getenv('api_key')

In [2]:
llm = ChatOpenAI(
    api_key=api_key,
    model='gpt-4o', 
)

embedding = OpenAIEmbeddings(
    api_key=api_key,
    model='text-embedding-3-small'
)

c:\Users\Ai Sukmoren\chatbot\.venv\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [10]:
pdf = PdfReader(r'C:\Users\Ai Sukmoren\chatbot\Chapter1-DigitalCurrency_DavidLee.pdf')

In [11]:
# read text from pdf
raw_text = ''
for i, page in enumerate(pdf.pages):
    content = page.extract_text()
    if content:
        raw_text += content

In [12]:
# We need to split the text using Character Text Split such that it sshould not increse token size
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 800,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)
len(texts)

128

In [13]:
document_search = FAISS.from_texts(texts, embedding)

In [7]:
chain = load_qa_chain(llm, chain_type="stuff")

In [15]:
query = "Who made this paper"
docs = document_search.similarity_search(query)
chain.run(input_documents=docs, question=query)

c:\Users\Ai Sukmoren\Desktop\chatbot\.venv\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


'The paper titled “The Impact of Discrimination in the Workplace on Worker’s Health and Productivity” was made by Ai Sukmoren (61100447), Kittapat Lochindarat (61100466), and Nattapat Chotirawi (61100480).'

In [27]:
def process_pdf_to_faiss_search(pdf_path, embedding_model):
    # Read the PDF file
    pdf = PdfReader(pdf_path)
    
    # Extract text from the PDF
    raw_text = ''
    for page in pdf.pages:
        content = page.extract_text()
        if content:
            raw_text += content
    
    # Split the text using Character Text Splitter
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=800,
        chunk_overlap=200,
        length_function=len,
    )
    texts = text_splitter.split_text(raw_text)
    
    # Create FAISS document search
    document_search = FAISS.from_texts(texts, embedding_model)
    
    # Save the FAISS index locally
    document_search.save_local("faiss_index")

In [28]:
pdf = r'C:\Users\Ai Sukmoren\chatbot\data\The Impact of Discrimination in the Workplace-DESKTOP-UB36V5M.pdf'
faiss = process_pdf_to_faiss_search(pdf,embedding)

In [9]:
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain

# Load the FAISS index with allow_dangerous_deserialization set to True
index_path = r"C:\Users\Ai Sukmoren\chatbot\faiss_index"
document_search = FAISS.load_local(index_path, embedding, allow_dangerous_deserialization=True)

# Define your query
query = "what is bitcoin"

# Perform a similarity search
docs = document_search.similarity_search(query)

# Load a QA chain
chain = load_qa_chain(llm, chain_type="stuff")

# Run the QA chain
answer = chain.run(input_documents=docs, question=query)

print("Answer:", answer)


c:\Users\Ai Sukmoren\chatbot\.venv\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Answer: Bitcoin is a decentralized digital currency, without a central bank or single administrator, that can be sent from user to user on the peer-to-peer bitcoin network without the need for intermediaries. Transactions are verified by network nodes through cryptography and recorded in a public distributed ledger called a blockchain. Bitcoin was invented in 2008 by an unknown person or group of people using the name Satoshi Nakamoto. The currency began use in 2009 when its implementation was released as open-source software.


In [ ]:
def gen_rag_answer(query:str)->str:
    # Load the FAISS index with allow_dangerous_deserialization set to True
    index_path = r"C:\Users\(Ai)AiSukmoren\Desktop\KMITL-present\faiss_index"
    document_search = FAISS.load_local(index_path, embedding, allow_dangerous_deserialization=True)
    
    # Perform a similarity search
    docs = document_search.similarity_search(query)

    # Load a QA chain
    chain = load_qa_chain(llm, chain_type="stuff")

    # Run the QA chain
    answer = chain.run(input_documents=docs, question=query)

    return answer

In [ ]:
res = gen_rag_answer('what the section olar Energy Offerings says')

In [ ]:
print(res)

In [2]:

from langchain_core.prompts import PromptTemplate
from langchain_community.tools import YouTubeSearchTool
import ast
import streamlit as st
import re

c:\Users\Ai Sukmoren\Desktop\chatbot\.venv\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [3]:
def get_yotube_link(ques):
        youtube = YouTubeSearchTool()
        result = youtube.run(ques)
        # Convert the string representation of the list back into a Python list
        result_list = ast.literal_eval(result)

        # Extract the first link
        first_link = result_list[0]
        
        # Extract video ID from the URL
        video_id = re.search(r'(?<=v=)[^&#]+', first_link).group()

        # Construct the thumbnail URL
        thumbnail_url = f'https://img.youtube.com/vi/{video_id}/0.jpg'

        # Display the thumbnail using Streamlit
        st.image(thumbnail_url)

        # Use a markdown to display a clickable link with custom text
        display_text = "link"  # Custom text for the link

        response = f"""
        here your url [{display_text}]({first_link}) !
        """
        return response